In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import os
import cv2

print(tf.VERSION)
print(tf.keras.__version__)

# pull in vid_to_20 from savedFramesIn20
# might not need to if videos are preprocessed

tf.enable_eager_execution()

file_path = "Images-Videos/"

1.13.1
2.2.4-tf


In [5]:
TestFrames1DIR = 'C:/Users/Fletcher/Documents/McDaniel/Summer 2019 research/Python Code/Images-Videos/'

CATEGORIES = ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching',
              'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair',
              'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'Breaststroke',
              'BrushingTeeth', 'CleanandJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen',
              'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl',
              'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking',
              'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow',
              'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges',
              'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing',
              'PlayingGuitar', 'PlayingPiano', 'PlayingTabla', 'PlayingViolin', 'PlayingCello', 'PlayingDaf',
              'PlayingDhol', 'PlayingFlute', 'PlayingSitar', 'PoleVault', 'PommelHorse', 'PullUps', 'Punch',
              'PushUps', 'Rafting', 'RockClimbingIndoor', 'RopeClimbing', 'Rowing', 'SalsaSpin', 'ShavingBeard',
              'Shotput', 'SkateBoarding', 'Skiing', 'Skijet', 'SkyDiving', 'SoccerJuggling', 'SoccerPenalty',
              'StillRings', 'SumoWrestling', 'Surfing', 'Swing', 'TableTennisShot', 'TaiChi', 'TennisSwing',
              'ThrowDiscus', 'TrampolineJumping', 'Typing', 'UnevenBars', 'VolleyballSpiking', 'WalkingWithDog',
              'WallPushups', 'WritingOnBoard', 'YoYo']

In [6]:
testing_data1 = []
catNum = 0
count = 0

for category in CATEGORIES:
    # limit to a certain number of categories, later use all categories
    if(catNum > 9):
        break
    
    pathCat = os.path.join(TestFrames1DIR, category)
    pathVid = TestFrames1DIR + str(category)
    print(category)
    
    for video in os.listdir(pathVid): 
        pathImg = pathVid + '/' + str(video)
        #print(video)
        
        frame_set = []
        
        for img in os.listdir(pathImg):
            img_array = cv2.imread(pathImg + '/' + img)
            
            frame_set.append([np.array(img_array), catNum])
            
        testing_data1.append([np.array(frame_set), catNum])
            
    catNum += 1

ApplyEyeMakeup


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Fletcher/Documents/McDaniel/Summer 2019 research/Python Code/Images-VideosApplyEyeMakeup'

In [ ]:
X = []
y = []

for features, label in testing_data1:
    X.append(features)
    y.append(label)

X = np.array(X)
print(X.shape)
print(y[1])

In [ ]:
X /= 255.0
X.shape

In [ ]:
X[0].size

In [4]:
# input tensor for the CNN
input1 = tf.keras.Input( shape=(227,227,3,) )
input1

<tf.Tensor 'input_1:0' shape=(?, 227, 227, 3) dtype=float32>

# testing adding the 20 frames

## almost working

In [ ]:
# first convolutional layer
# !!! kinda confused if 'filters' parameter is number of filters or size of each. assumed it is number. !!!
# should each output shape 55x55x96

frames = []

for entry in X:    # the 20 frame array
    for frame in entry:
        frame_ = tf.keras.layers.Conv2D(96, 11, input_shape=(3, 227, 227), strides=4, activation='relu' )(frame)
        frames += frame_


# add the 20 frames into each other
# should output shape 55x55x96 
added = tf.keras.layers.Add()( frames )

## this is the rest of the network, but can't test until input works

In [8]:
# normalization
norm1 = layers.LayerNormalization()( added )
# first pooling layer
pool1 = layers.MaxPool2D( pool_size=3, strides=2 )( norm1 )

# first 2D convolutional
conv2 = layers.Conv2D( filters=23, kernel_size=256, strides=1 )( pool1 )
# normalization
norm2 = layers.LayerNormalization()(conv2)
# second pooling
pool2 = layers.MaxPool2D( pool_size=3, strides=2 )( norm2 )

# third convolutional layer
conv3 = layers.Conv2D( filters=9, kernel_size=384, strides=1 )( pool2 )
# fourth convolutional
conv4 = layers.Conv2D( filters=7, kernel_size=384, strides=1 )( conv3 )
# fifth convolutional
conv5 = layers.Conv2D( filters=5, kernel_size=256, strides=1 )( conv4 )

# third pooling
pool3 = layers.MaxPool2D( pool_size=3, strides=2 )( conv5 )

# 
#model.add(Flatten())

# three fully-connected layers
fc1 = layers.Dense( units=2048 )( pool3 )
fc2 = layers.Dense( units=2048 )( fc1 )
fc3 = layers.Dense( units=101 )( fc2 )

# softmax
soft = layers.Activation( 'softmax' )( fc3 )

model = tf.keras.Model( inputs=inputs, outputs=soft )

AttributeError: module 'tensorflow._api.v1.keras.layers' has no attribute 'LayerNormalization'